In [21]:
# This code reads in a lensing galaxy catalogue and appends to it 
# values of the mass over-density and tidal field tensor at the galaxy locations
# By Joachim Harnois-Deraps, 2021.

import numpy as np
import healpy as hp
#from scipy.interpolate import interp1d
import random
import input
from pathlib import Path


# The tomographic bin
tomobin = 1

print("**************************")
print("Processing tomo bin",tomobin)
print("**************************")



# the number of galaxies per square arcminute to extract
# Only needed to choose the correct galaxy catalogue
gpam = 0.06

# The NSIDE value of the shear and weight maps
nside = 8192

npix = 12*nside**2
pix_size = np.sqrt(4.*np.pi*(180./np.pi*60)**2/npix)
print('pixel size of shear maps is %f arcmin per side' % pix_size)

zfile=np.loadtxt("z2ts.txt",delimiter=',')
z_list=np.flip(zfile[1:58,0])
snaplist = np.flip(zfile[1:58,1].astype(int))
n_slices = np.size(z_list)


**************************
Processing tomo bin 5
**************************
pixel size of shear maps is 0.429436 arcmin per side


In [23]:
#for plane in range(0,41):
for plane in range(0,n_slices):

    print('starting analysis of plane %i' % plane)
            
    fname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w.asc'
    # check file exists:
    my_file = Path(fname)
    if my_file.is_file():
        # file exists
        print("Got file! Reading",fname)
        ra, dec, shear1, shear2, w, z = np.loadtxt(fname, unpack=True)
        
        #load mass maps and tidal tensor
        fname = input.deltaDir+"density_map_"+str(snaplist[plane])+"_dens_allsky.npy"
        print("opening ", fname) 
        hpmap_density = np.load(fname)
        print("Got density file!")
        
        fname = input.tidalDir+"tidal_field_map_"+str(snaplist[plane])+"_allsky.npy"
        print("opening ", fname) 
        hpmap_tidal = np.load(fname)
        print("Got tidal file!")
        

        # Normalize the maps correctly:       
        mean_rho = (np.mean(hpmap_density)) # no factor of 8.0 since we have full sky data here
        hpmap_density /= mean_rho 
        hpmap_density -= 1
        
        #shell thickness, in cells
        nc_shell=114/4225.35*20480
        
        hpmap_tidal /= nc_shell

        print("over density:",np.min(hpmap_density), np.max(hpmap_density), np.mean(hpmap_density))
        print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
        print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
        print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))


        
        # Interpolate maps at galaxy positions
        den_pix = hp.get_interp_val(hpmap_density, (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )

        s11_pix = hp.get_interp_val(hpmap_tidal[:,0], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s22_pix = hp.get_interp_val(hpmap_tidal[:,1], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s12_pix = hp.get_interp_val(hpmap_tidal[:,2], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )



        # Save a shear catalogue in standard format to be read by Athena
        outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'

        print("Wrinting",outname)

        # given ra=np.degrees(phi), e2 DOES NOT NEED FLIPPING.
        #np.savetxt(outname, np.c_[ra_rand, dec_rand, shear1_pix_front*dz1 + shear1_pix_back*dz2, shear2_pix_front*dz1 + shear2_pix_back*dz2, np.ones(ngal_cur[plane]),spec_z[gal_cur],kappa_pix_front*dz1 + kappa_pix_back*dz2 ])
        np.savetxt(outname, np.c_[ra, dec, shear1, shear2, w,z, den_pix, s11_pix, s22_pix, s12_pix])



    else:
        print('found no file named '+fname)


print("Done!")

starting analysis of plane 0
Got file! Reading IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo5_plane0_0.06GpAM_RA_Dec_g1_g2_w.asc
opening  IA-infusion/SkySim5000/density/density_map_121_dens_allsky.npy


/tmp/ipykernel_1198/1111463654.py:6: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w.asc'


Got density file!
opening  IA-infusion/SkySim5000/tidalfield/tidal_field_map_121_allsky.npy
Got tidal file!
over density: -0.94428676 8.554813 5.5272477e-07
s11: 0.0 0.0036337813 2.3507203e-06
s22: 0.0 0.004895614 2.3575897e-06
s12: 0.0 0.0041355877 6.7045307e-06


/tmp/ipykernel_1198/1111463654.py:53: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'


Wrinting IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo5_plane0_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.asc
starting analysis of plane 1
Got file! Reading IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo5_plane1_0.06GpAM_RA_Dec_g1_g2_w.asc
opening  IA-infusion/SkySim5000/density/density_map_124_dens_allsky.npy
Got density file!
opening  IA-infusion/SkySim5000/tidalfield/tidal_field_map_124_allsky.npy
Got tidal file!
over density: -0.912908 9.160751 -1.8629972e-06
s11: 0.0 0.004117764 2.500499e-06
s22: 0.0 0.0044258796 2.502276e-06
s12: 0.0 0.003699286 7.111228e-06
Wrinting IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo5_plane1_0.06GpAM_RA_Dec_g1_g2_w_den_Sij.asc
starting analysis of plane 2
Got file! Reading IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo5_plane2_0.06GpAM_RA_Dec_g1_g2_w.asc
opening  IA-infusion/SkySim5000/density/density_map_127_dens_allsky.npy
Got density file!
opening  IA-infusion/SkySim5000/tidalfield/tidal_field_map_127_allsky.npy
Got tidal file!
over 

In [7]:
   random_vector = [[ 0.4854 , 0.8003],
    [0.1419  ,  0.4218],
    [0.9157  ,  0.7922],
    [0.9595  ,  0.6557]]


In [9]:
for plane in range(0,n_slices):
    print(plane, snaplist[plane], )

0 121
1 124
2 127
3 131
4 134
5 137
6 141
7 144
8 148
9 151
10 155
11 159
12 163
13 167
14 171
15 176
16 180
17 184
18 189
19 194
20 198
21 203
22 208
23 213
24 219
25 224
26 230
27 235
28 241
29 247
30 253
31 266
32 272
33 279
34 286
35 293
36 300
37 307
38 315
39 323
40 331
41 338
42 347
43 355
44 365
45 373
46 382
47 392
48 401
49 411
50 421
51 432
52 442
53 453
54 464
55 475
56 487


In [11]:
random_vector[:][0]

[0.4854, 0.8003]